## Modeling 

**Name**: Diane Lu

**Contact**: dianengalu@gmail.com

**Date**: 07/31/2023

### Table of Contents 

1. [Introduction](#intro)
2. [Model Dataset](#model)
    * Data Dictionary
3. [Collaborative-Filtering Recommendation System without SVD](#nosvd)
4. [Collaborative-Filtering Recommendation System with SVD](#svd)
5. [Collaborative-Filtering Recommendation System with FunkSVD](#funksvd)

### Introduction <a class="anchor" id="intro"></a>

During the Initial Modeling stage, we create the first version of the restaurant recommendation system, which will serve as our starting point for future improvements and enhancements.

#### Importing Python Libraries 

Importing necessary libraries.

In [1]:
# Import necessary libraries
import numpy as np 
import pandas as pd 

# Import cosine_similarity function from scikit-learn
from sklearn.metrics.pairwise import cosine_similarity

# Import SVD algorithm from Surprise library
from surprise import SVD

# Import Reader and Dataset from Surprise library
from surprise.reader import Reader
from surprise import Dataset

# Import FunkSVD algorithm from Surprise library
from surprise.prediction_algorithms.matrix_factorization import SVD as FunkSVD

# Import train_test_split and GridSearchCV from Surprise library
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

# Import accuracy module from Surprise library
from surprise import accuracy

# Ignore all warnings to avoid cluttering the output
import warnings
warnings.filterwarnings("ignore")

### Model Dataset <a class="anchor" id="model"></a>

**Data Dictionary:**
* `user_id`: unique user id
* `business_id`: unique user id
* `rating`: star rating

In [2]:
# Read data from a pickle file into a Pandas DataFrame
vancouver_data = pd.read_pickle('/Users/diane/Desktop/BrainStation/Brainstation_Capstone/yelp_data/vancouver_data.pkl')


In [3]:
# Display concise information about the 'vancouver_data' DataFrame
vancouver_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64660 entries, 1101 to 5561981
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          64660 non-null  int64  
 1   business_id      64660 non-null  int64  
 2   rating           64660 non-null  float64
 3   restaurant_name  64660 non-null  object 
 4   categories       64660 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 3.0+ MB


In [4]:
# Display the first few rows of the 'vancouver_data' DataFrame
vancouver_data.head()

,user_id,business_id,rating,restaurant_name,categories
1101,70315,1407,4.0,Meat & Bread,"[Fast Food, Bakeries, Sandwiches, Salad, Soup,..."
1105,70315,1356,3.0,Edible Canada At the Market,"[Seafood, Canadian (New), American (New), Spec..."
1109,70315,7370,4.0,The Lamplighter Public House,"[Nightlife, Gastropubs, Bars, Pubs]"
1144,70315,1143,5.0,Miku,"[Japanese, Sushi Bars]"
1151,70315,13469,4.0,Lupo,[Italian]


In [5]:
# Count the number of missing values in each column of the 'vancouver_data' DataFrame
vancouver_data.isnull().sum()

user_id            0
business_id        0
rating             0
restaurant_name    0
categories         0
dtype: int64

In [6]:
# Print the size of our model dataset
print(f"The size of our model dataset is {vancouver_data.shape[0]} entries.")

The size of our model dataset is 64660 entries.


In [7]:
# Extract columns 'user_id', 'restaurant_name', and 'rating' from 'vancouver_data',
# then sort the data by 'user_id' in ascending order
sorted_data = vancouver_data[['user_id', 'restaurant_name', 'rating']].sort_values(by='user_id')

# Display the sorted data
display(sorted_data)

,user_id,restaurant_name,rating
2328038,4,Breakfast Table,2.0
2328033,4,Yolks,5.0
2328050,4,Fable,3.0
2328052,4,Minami,5.0
2328053,4,The Flying Pig - Gastown,4.0
...,...,...,...
1342235,81124,The Sandbar Seafood Restaurant,5.0
1342237,81124,The Flying Pig - Yaletown,1.0
1342238,81124,Black Rice Izakaya,2.0
1999029,81139,Marutama Ramen,5.0


In [8]:
# Get unique user_id values and map them to new values starting from 0
user_id_mapping = {user_id: new_id for new_id, user_id in enumerate(sorted_data['user_id'].unique())}

# Replace the 'user_id' values in the DataFrame using the mapping
sorted_data['user_id'] = sorted_data['user_id'].map(user_id_mapping)

# Display the updated DataFrame
display(sorted_data)

,user_id,restaurant_name,rating
2328038,0,Breakfast Table,2.0
2328033,0,Yolks,5.0
2328050,0,Fable,3.0
2328052,0,Minami,5.0
2328053,0,The Flying Pig - Gastown,4.0
...,...,...,...
1342235,8976,The Sandbar Seafood Restaurant,5.0
1342237,8976,The Flying Pig - Yaletown,1.0
1342238,8976,Black Rice Izakaya,2.0
1999029,8977,Marutama Ramen,5.0


In [9]:
# Number of restaurants 
print("Number of restaurants:", sorted_data['restaurant_name'].nunique())

# Number of unique reviewers 
print("Number of unique reviewers:", sorted_data['user_id'].nunique())

# Range of ratings
print("Range of ratings:", sorted_data['rating'].min(), "to", sorted_data['rating'].max())

Number of restaurants: 766
Number of unique reviewers: 8978
Range of ratings: 1.0 to 5.0


### Building the Matrixes for the Model

In [10]:
def create_user_item_matrix(data):
    # Create the User-Item Matrix
    user_item_matrix = data.pivot_table(index='user_id', columns='restaurant_name', values='rating')

    return user_item_matrix

In [11]:
# Create the user-item matrix using the create_user_item_matrix function
user_item_matrix = create_user_item_matrix(sorted_data)

# Display the user-item matrix
display(user_item_matrix)

restaurant_name,3G Vegetarian Restaurant,49th Parallel Coffee,6 Degrees Eatery,A La Mode,ARC Restaurant,Abigail's Party,Absinthe Bistro,Acacia Fillo Bar,Acme Cafe,Adesso Bistro,...,Yaletown Brewing Company,Yamato Sushi Restaurant,Yolk's Breakfast,Yolks,Yui Japanese Bistro,Zabu Chicken,Zakkushi Dining On Main,Zakkushi on Denman,Zefferelli's,Zeitoon Restaurant
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
8974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Collaborative-Filtering Recommendation System with FunkSVD <a class="anchor" id="funksvd"></a>

FunkSVD is a specific variant of SVD designed for collaborative filtering tasks in recommendation systems. It addresses the sparsity issue present in user-item interaction matrices by incorporating stochastic gradient descent to handle missing values efficiently. FunkSVD performs matrix factorization and decomposes the user-item interaction matrix into user and item latent feature matrices.

In [12]:
# Load your user-item interaction data into Surprise Dataset
reader = Reader(rating_scale=(1, 5))

sur_data = Dataset.load_from_df(sorted_data[['user_id', 'restaurant_name', 'rating']], reader)

In [13]:
# Split the data into training and testing sets
trainset, testset = train_test_split(sur_data, test_size=0.2, random_state=42)

In [14]:
# Build and train the FunkSVD-based collaborative filtering model
model = FunkSVD(n_factors=50, biased=True, random_state=42)
model.fit(trainset)

In [15]:
# Make predictions on the test set
predictions = model.test(testset)

In [17]:
# Calculate and print Root Mean Squared Error (RMSE)
RMSE = accuracy.rmse(predictions, verbose=False)
print("Root Mean Squared Error (RMSE):", RMSE)

# Calculate and print Mean Squared Error (MSE)
MSE = accuracy.mse(predictions, verbose=False)
print("Mean Squared Error (MSE):", MSE)

Root Mean Squared Error (RMSE): 0.9191623580796869
Mean Squared Error (MSE): 0.8448594405106107


In [18]:
# Calculate Fraction of Concordant Pairs (FCP)
FCP = accuracy.fcp(predictions, verbose=False)

# Print the Fraction of Concordant Pairs (FCP) score
print("Fraction of Concordant Pairs (FCP):", FCP)

Fraction of Concordant Pairs (FCP): 0.5972611285873871


In [19]:
# Example Usage: Recommend restaurants similar to a specific restaurant
restaurant_name = "Miku" 

In [20]:
# Get the user-item matrix used for factorization
trainset_full = sur_data.build_full_trainset()
user_item_matrix = trainset_full.ur

In [21]:
# Find the index of the input restaurant name in the pivot table
restaurant_index = trainset_full.to_inner_iid(restaurant_name)

In [22]:
# Get the latent factors for the input restaurant
restaurant_factors = model.qi[restaurant_index]

In [23]:
# Calculate similarity scores with other restaurants based on latent factors
similarity_scores = np.dot(model.qi, restaurant_factors)

In [24]:
# Sort the restaurants based on similarity scores in descending order
similar_restaurant_indices = np.argsort(similarity_scores)[::-1]

In [25]:
# Get top N recommended restaurants (excluding the input restaurant itself)
top_n = 5
recommended_restaurants = []
for index in similar_restaurant_indices:
    name = trainset_full.to_raw_iid(index)
    if name != restaurant_name:
        recommended_restaurants.append(name)
        if len(recommended_restaurants) == top_n:
            break

print("Recommended Restaurants for {}: {}".format(restaurant_name, recommended_restaurants))

Recommended Restaurants for Miku: ['Memphis Blues Barbeque House', 'JOEY Bentall One', 'Adesso Bistro', 'So Hyang Korean Cuisine', 'Tom Sushi']


### Model Evaluation of Recommendation System with FunkSVD  <a class="anchor" id="funksvd"></a>

FunkSVD is a specific variant of SVD designed for collaborative filtering tasks in recommendation systems. It addresses the sparsity issue present in user-item interaction matrices by incorporating stochastic gradient descent to handle missing values efficiently. FunkSVD performs matrix factorization and decomposes the user-item interaction matrix into user and item latent feature matrices.

#### Hyperparameter Optimization through GridSearchCV



In [26]:
# Set the parameter grid
param_grid = {
    'n_factors': [100, 150], 
    'n_epochs': [10, 20],
    'lr_all': [0.005, 0.1],
    'biased': [False] } #The parameter indicates to the algorithm that all latent information must be stored. 

# Set GridSearchCV with 3 cross validation
GS = GridSearchCV(FunkSVD, param_grid, measures=['fcp'], cv=3)

# Fit the model
GS.fit(sur_data)

In [27]:
# Get the best FCP accuracy score from GridSearchCV results
gsbs = GS.best_score['fcp']

# Get the best parameters for FCP from GridSearchCV results
gsbp = GS.best_params['fcp']

# Check the FCP accuracy score (1.0 is ideal and 0 is worst)
print("FCP accuracy score:", gsbs)

# Check the best parameters for FCP
print("Best parameters for FCP:", gsbp)

FCP accuracy score: 0.6437490104268117
Best parameters for FCP: {'n_factors': 150, 'n_epochs': 10, 'lr_all': 0.005, 'biased': False}


#### Building Recommendation System with FunkSVD

In [28]:
# Split train test set
trainset, testset = train_test_split(sur_data, test_size=0.25)

# Set the algorithm
my_svd = FunkSVD(n_factors=150, 
                 n_epochs=10, 
                 lr_all=0.005, 
                 biased=False,
                 verbose=0)

# Fit train set
my_svd.fit(trainset)

# Test the algorithm using test set
my_pred = my_svd.test(testset)

In [29]:
# Put 'my_pred' results in a DataFrame
df_prediction = pd.DataFrame(my_pred, columns=['user_id',
                                               'restaurant_name',
                                               'actual',
                                               'prediction',
                                               'details'])

# Calculate the difference of actual and prediction into the 'diff' column
df_prediction['diff'] = abs(df_prediction['prediction'] - df_prediction['actual'])


#### Evaluating Predictions

In [30]:
# Check the df_prediction
df_prediction.head()

,user_id,restaurant_name,actual,prediction,details,diff
0,2572,Sal y Limón,4.0,3.446009,{'was_impossible': False},0.553991
1,8168,Nuba in Gastown,4.0,4.145937,{'was_impossible': False},0.145937
2,7460,La Cigale French Bistro,3.0,3.463098,{'was_impossible': False},0.463098
3,7228,Honey Salt,4.0,3.190576,{'was_impossible': False},0.809424
4,2422,JOEY Broadway,2.0,2.144667,{'was_impossible': False},0.144667


In [31]:
# See the best 10 predictions
df_prediction.sort_values(by='diff')[:10]

,user_id,restaurant_name,actual,prediction,details,diff
8018,7898,Guu Original Thurlow,5.0,5.0,{'was_impossible': False},0.0
2949,6772,Lupo,5.0,5.0,{'was_impossible': False},0.0
14668,7729,Guu Original Thurlow,5.0,5.0,{'was_impossible': False},0.0
8381,7066,GyuDonYa,1.0,1.0,{'was_impossible': False},0.0
15547,682,Marutama Ramen,5.0,5.0,{'was_impossible': False},0.0
4091,2119,Le Crocodile Restaurant,5.0,5.0,{'was_impossible': False},0.0
14658,3376,Water Street Cafe,1.0,1.0,{'was_impossible': False},0.0
13249,8003,The Gluten Free Epicurean,1.0,1.0,{'was_impossible': False},0.0
1147,2845,Marutama Ramen,5.0,5.0,{'was_impossible': False},0.0
7015,411,Jules French Bistro,1.0,1.0,{'was_impossible': False},0.0


In [32]:
# See the worst 10 predictions
df_prediction.sort_values(by='diff')[-10:]

,user_id,restaurant_name,actual,prediction,details,diff
203,2520,Tuc Craft Kitchen,5.0,1.0,{'was_impossible': False},4.0
13870,4667,The Irish Heather,5.0,1.0,{'was_impossible': False},4.0
1727,527,Guu Original Thurlow,5.0,1.0,{'was_impossible': False},4.0
5778,3259,Pourhouse Restaurant,5.0,1.0,{'was_impossible': False},4.0
13935,69,Fritz European Fry House,5.0,1.0,{'was_impossible': False},4.0
8341,4581,The Shameful Tiki Room,5.0,1.0,{'was_impossible': False},4.0
1675,1953,Giovane,5.0,1.0,{'was_impossible': False},4.0
211,6603,Steamworks Brewing Company,5.0,1.0,{'was_impossible': False},4.0
13817,4812,Hokkaido Ramen Santouka,5.0,1.0,{'was_impossible': False},4.0
4102,6711,Dynasty Seafood Restaurant,5.0,1.0,{'was_impossible': False},4.0


In [33]:
# Check total rows with same actual and prediction ratings
df_prediction[df_prediction['diff'] <= 0]

,user_id,restaurant_name,actual,prediction,details,diff
134,3492,Salt Tasting Room,5.0,5.0,{'was_impossible': False},0.0
153,2021,Pizza Garden Smithe,1.0,1.0,{'was_impossible': False},0.0
282,5349,Sushi Itoga,5.0,5.0,{'was_impossible': False},0.0
426,6772,Lupo,5.0,5.0,{'was_impossible': False},0.0
579,1825,Medina Cafe,5.0,5.0,{'was_impossible': False},0.0
...,...,...,...,...,...,...
15547,682,Marutama Ramen,5.0,5.0,{'was_impossible': False},0.0
15645,1499,Miku,5.0,5.0,{'was_impossible': False},0.0
15773,1444,Fritz European Fry House,5.0,5.0,{'was_impossible': False},0.0
16078,7128,Vij's Restaurant,5.0,5.0,{'was_impossible': False},0.0


In [34]:
(df_prediction['diff'] == 0).mean()

0.009774203526136714

In [35]:
(df_prediction["diff"] <= 1).mean()

0.5300958861738324

In [36]:
# Build full trainset
full_trainset = sur_data.build_full_trainset()

# Build the SVD algorithm
my_svd = FunkSVD(n_factors=150, 
                 n_epochs=10, 
                 lr_all=0.005, 
                 biased=False,
                 verbose=0)

# Fit with full trainset
my_svd.fit(full_trainset)

In [37]:
# Define the full test set
full_testset = full_trainset.build_anti_testset(fill=-1)

In [38]:
# Set the prediction
my_prediction = my_svd.test(full_testset)

In [39]:
# Put 'my_pred' results in a DataFrame
df_prediction = pd.DataFrame(my_pred, columns=['user_id',
                                               'restaurant_name',
                                               'actual',
                                               'prediction',
                                               'details'])

In [40]:
# Check user id `1497` predictions
df = df_prediction[df_prediction['user_id'] == 7857]\
    .sort_values(by=['prediction'], ascending=False)\
    .head()

display(df)

,user_id,restaurant_name,actual,prediction,details
7516,7857,Miku,5.0,4.666105,{'was_impossible': False}
14893,7857,Tojo's,5.0,4.464990,{'was_impossible': False}
4158,7857,Kokoro Tokyo Mazesoba,5.0,4.169717,{'was_impossible': False}
3862,7857,Las Margaritas Restaurant,5.0,2.978440,{'was_impossible': False}
5781,7857,MeeT in Yaletown,3.0,2.941351,{'was_impossible': False}


#### Recommendation System Model Evaluation 

In [41]:
# The surprise package doesn't allow you to test on the trainset we built
my_train_dataset, my_test_dataset = train_test_split(sur_data, test_size=0.5)

predictions = my_svd.test(my_test_dataset)

In [42]:
# Calculate and print Root Mean Squared Error (RMSE)
RMSE = accuracy.rmse(predictions, verbose=False)
print("Root Mean Squared Error (RMSE):", RMSE)

# Calculate and print Mean Squared Error (MSE)
MSE = accuracy.mse(predictions, verbose=False)
print("Mean Squared Error (MSE):", MSE)

Root Mean Squared Error (RMSE): 0.9892428385166391
Mean Squared Error (MSE): 0.9786013935564574


In [43]:
# Calculate Fraction of Concordant Pairs (FCP)
FCP = accuracy.fcp(predictions, verbose=False)

# Print the Fraction of Concordant Pairs (FCP) score
print("Fraction of Concordant Pairs (FCP):", FCP)

Fraction of Concordant Pairs (FCP): 0.7374532705664781
